# Requirement

In [17]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import brentq
from scipy.stats import norm
from datetime import datetime, timedelta
import requests
import json
from polygon import RESTClient

# Functions

In [18]:
# Black-Scholes formula for calls and puts
def bs_call_price(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)

def bs_put_price(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r*T) * norm.cdf(-d2) - S * norm.cdf(-d1)

# Implied volatility calculation using Brent's method
def implied_vol_call(market_price, S, K, T, r):
    def obj_func(sigma):
        return bs_call_price(S, K, T, r, sigma) - market_price
    try:
        return brentq(obj_func, 1e-6, 5)
    except ValueError:
        return np.nan

def implied_vol_put(market_price, S, K, T, r):
    def obj_func(sigma):
        return bs_put_price(S, K, T, r, sigma) - market_price
    try:
        return brentq(obj_func, 1e-6, 5)
    except ValueError:
        return np.nan


def atm_option(symbol, date):


    api_key = "XN1r3nHQ1Rb3SsbqdwDI72dE35vJkCzP"
    client = RESTClient(api_key)

    date_str = date.strftime("%Y-%m-%d")
    # Get the close price on the given date
    aggs = client.get_aggs(symbol, 1, "day", date_str, date_str, limit=1)
    if not aggs:
        print("No historical data for the given date.")
        return None, None
    close_price = aggs[0].close
    print(close_price)

    # Get options contracts as of the given date
    contracts = list(client.list_options_contracts(underlying_ticker=symbol, as_of=date_str, limit=1000))
    if not contracts:
        print("No options contracts found.")
        return None, None

    # Find target expiration date (approx 30 days from given date)
    target_exp = date + timedelta(days=30)
    unique_exps = set(c.expiration_date for c in contracts if c.expiration_date > date_str)
    if not unique_exps:
        print("No future expiration dates found.")
        return None, None

    exp_diffs = [(abs((datetime.strptime(e, '%Y-%m-%d') - target_exp).days), e) for e in unique_exps]
    best_diff, exp_date = min(exp_diffs)
    print(f"Selected expiration: {exp_date} (diff: {best_diff} days)")

    # Filter calls and puts for that expiration
    calls = [c for c in contracts if c.expiration_date == exp_date and c.contract_type == 'call']
    puts = [c for c in contracts if c.expiration_date == exp_date and c.contract_type == 'put']

    if not calls or not puts:
        print("No calls or puts found for the selected expiration.")
        return None, None

    # Find ATM call and put (closest strike to close_price)
    atm_call = min(calls, key=lambda c: abs(c.strike_price - close_price))
    atm_put = min(puts, key=lambda c: abs(c.strike_price - close_price))

    print(f"ATM Call: {atm_call.ticker} (strike: {atm_call.strike_price})")
    print(f"ATM Put: {atm_put.ticker} (strike: {atm_put.strike_price})")

    return atm_call.ticker, atm_put.ticker

    




# Beginning

In [ ]:
symbol = "NVDA"
ticker = yf.Ticker(symbol)
r = 0.045
atm_call, atm_put = atm_option(symbol, datetime.strptime('2024-11-05', "%Y-%m-%d").date())

print(atm_call)
print(atm_put)

139.91


In [ ]:


# # Implied Volatility for ATM Call
# iv_atm_call = implied_vol_call(atm_call['lastPrice'], s0, atm_call['strike'], atm_call['ttm'], r)
# print(f"Implied volatility (ATM Call): {iv_atm_call:.4f}")

# # Implied Volatility for ATM Put
# iv_atm_put = implied_vol_put(atm_put['lastPrice'], s0, atm_put['strike'],atm_put['ttm'], r)
# print(f"Implied volatility (ATM Put): {iv_atm_put:.4f}")

# # 20-day historical volatility of underlying
# close_prices = ticker.history(period="30d")['Close']
# returns = close_prices.pct_change().dropna()
# hist_vol_20d = returns[-20:].std() * np.sqrt(252)

# print(f"20-day annualized volatility of underlying: {hist_vol_20d:.4f}")



Implied volatility (ATM Call): nan
Implied volatility (ATM Put): 0.1017
20-day annualized volatility of underlying: 0.3164


In [ ]:
class Agent:
    def __init__(self,balance):
        self.balance = balance
        
            